In [153]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import math

In [154]:
# Jupyter configuration
pd.set_option('display.max_rows', 500)

In [155]:
dataset = pd.read_csv('owid-covid-data.csv')

Separate the data into two datasets

In [156]:
countries_dataset = dataset[dataset['location'] != 'World']
world_dataset = dataset[dataset['location'] == 'World']

countries_dataset.to_csv('countries-covid-data.csv')
world_dataset.to_csv('world-covid-data.csv')

dataset = countries_dataset


In [157]:
# The shape of the dataset
dataset.shape

(127167, 65)

### Variables description

| Variable                             | Description                                                                                                    |
|:-------------------------------------|:---------------------------------------------------------------------------------------------------------------|
| `icu_patients`                       | Number of COVID-19 patients in intensive care units (ICUs) on a given day                                      |
| `icu_patients_per_million`           | Number of COVID-19 patients in intensive care units (ICUs) on a given day per 1,000,000 people                 |
| `hosp_patients`                      | Number of COVID-19 patients in hospital on a given day                                                         |
| `hosp_patients_per_million`          | Number of COVID-19 patients in hospital on a given day per 1,000,000 people                                    |
| `weekly_icu_admissions`              | Number of COVID-19 patients newly admitted to intensive care units (ICUs) in a given week                      |
| `weekly_icu_admissions_per_million`  | Number of COVID-19 patients newly admitted to intensive care units (ICUs) in a given week per 1,000,000 people |
| `weekly_hosp_admissions`             | Number of COVID-19 patients newly admitted to hospitals in a given week                                        |
| `weekly_hosp_admissions_per_million` | Number of COVID-19 patients newly admitted to hospitals in a given week per 1,000,000 people                   |

### Policy responses
| Variable           | Description                                                                                                                                                                                                         |
|:-------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `stringency_index` | Government Response Stringency Index: composite measure based on 9 response indicators including school closures, workplace closures, and travel bans, rescaled to a value from 0 to 100 (100 = strictest response) |

### Reproduction rate
| Variable            | Description                                                                                                                                   |
|:--------------------|:----------------------------------------------------------------------------------------------------------------------------------------------|
| `reproduction_rate` | Real-time estimate of the effective reproduction rate (R) of COVID-19. See https://github.com/crondonm/TrackingR/tree/main/Estimates-Database |

### Tests & positivity
| Variable                          | Description                                                                                                                                                                                                                                                                                                          |
|:----------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `total_tests`                     | Total tests for COVID-19                                                                                                                                                                                                                                                                                             |
| `new_tests`                       | New tests for COVID-19 (only calculated for consecutive days)                                                                                                                                                                                                                                                        |
| `total_tests_per_thousand`        | Total tests for COVID-19 per 1,000 people                                                                                                                                                                                                                                                                            |
| `new_tests_per_thousand`          | New tests for COVID-19 per 1,000 people                                                                                                                                                                                                                                                                              |
| `new_tests_smoothed`              | New tests for COVID-19 (7-day smoothed). For countries that don't report testing data on a daily basis, we assume that testing changed equally on a daily basis over any periods in which no data was reported. This produces a complete series of daily figures, which is then averaged over a rolling 7-day window |
| `new_tests_smoothed_per_thousand` | New tests for COVID-19 (7-day smoothed) per 1,000 people                                                                                                                                                                                                                                                             |
| `positive_rate`                   | The share of COVID-19 tests that are positive, given as a rolling 7-day average (this is the inverse of tests_per_case)                                                                                                                                                                                              |
| `tests_per_case`                  | Tests conducted per new confirmed case of COVID-19, given as a rolling 7-day average (this is the inverse of positive_rate)                                                                                                                                                                                          |
| `tests_units`                     | Units used by the location to report its testing data 

Looking at these variables one can observe that for some variables there are pair variables that end wirh 'per_thousand' or 'per_million'. These variables generalize the information regarding the variable that is represented by tehm. We can exclude these variables because they have the same behaviour as the real measurements. For example, icu_patients in a way represents the same information as variable icu_patients_per_million and should have the same behaviour. The variables to be excluded are: 

* icu_patients_per_million
* hosp_patients_per_million
* weekly_icu_admissions_per_million
* weekly_hosp_admissions_per_million
* total_tests_per_thousand
* new_tests_per_thousand
* new_tests_smoothed
* new_tests_smoothed_per_thousand.

In [158]:
dataset.drop([
        'icu_patients_per_million',
        'hosp_patients_per_million', 
        'weekly_icu_admissions_per_million',
        'weekly_hosp_admissions_per_million',
        'total_tests_per_thousand',
        'new_tests_per_thousand',
        'new_tests_smoothed',
        'new_tests_smoothed_per_thousand'
    ],
    axis='columns',
    inplace=True
)

dataset.shape

(127167, 57)

Select only a subset of variables to work with

In [159]:
dataset = dataset[[
    'location',
    'date',
    'icu_patients',
    'hosp_patients',
    'weekly_icu_admissions', 
    'weekly_hosp_admissions', 
    'stringency_index', 
    'reproduction_rate', 
    'total_tests', 
    'new_tests', 
    'positive_rate', 
    'tests_per_case', 
    'tests_units'
]]

dataset.shape

(127167, 13)

In [160]:
# All available variables
dataset.columns.tolist()

['location',
 'date',
 'icu_patients',
 'hosp_patients',
 'weekly_icu_admissions',
 'weekly_hosp_admissions',
 'stringency_index',
 'reproduction_rate',
 'total_tests',
 'new_tests',
 'positive_rate',
 'tests_per_case',
 'tests_units']

In [161]:
# Fixing date column datatype
dataset['date'] = pd.to_datetime(dataset['date'])

/tmp/ipykernel_9921/1142007443.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['date'] = pd.to_datetime(dataset['date'])


In [162]:
dataset.describe()

,icu_patients,hosp_patients,weekly_icu_admissions,weekly_hosp_admissions,stringency_index,reproduction_rate,total_tests,new_tests,positive_rate,tests_per_case
count,15022.000000,17713.000000,1262.000000,2186.000000,106613.000000,101931.000000,5.469900e+04,5.450800e+04,61972.000000,61316.000000
mean,915.236786,4034.903066,214.015366,2976.750189,56.498595,0.999924,1.160052e+07,5.647572e+04,0.086487,159.830328
std,2879.425099,11403.399705,494.954694,10333.090304,20.648320,0.341983,4.843986e+07,1.950166e+05,0.095893,841.823266
min,0.000000,0.000000,0.000000,0.000000,0.000000,-0.030000,0.000000e+00,1.000000e+00,0.000000,1.000000
25%,25.000000,100.000000,5.091000,43.085000,42.590000,0.830000,2.506645e+05,2.098000e+03,0.016000,8.000000
50%,122.000000,488.000000,25.856500,250.761000,57.410000,1.010000,1.262217e+06,7.641500e+03,0.050000,19.500000
75%,525.750000,2278.000000,150.833500,1206.866500,72.220000,1.170000,5.162169e+06,2.950950e+04,0.124000,57.800000
max,28891.000000,133253.000000,4002.456000,116307.000000,100.000000,5.960000,6.215662e+08,3.740296e+06,0.970000,50000.000000


In [163]:
# Variable types
dataset.dtypes

location                          object
date                      datetime64[ns]
icu_patients                     float64
hosp_patients                    float64
weekly_icu_admissions            float64
weekly_hosp_admissions           float64
stringency_index                 float64
reproduction_rate                float64
total_tests                      float64
new_tests                        float64
positive_rate                    float64
tests_per_case                   float64
tests_units                       object
dtype: object

#### There are two variables that are not numerical: date and tests_units. 

### Before dealing with missing values, let's take a look at the head and tail of the resulting dataset. 

In [164]:
dataset.head(10)

,location,date,icu_patients,hosp_patients,weekly_icu_admissions,weekly_hosp_admissions,stringency_index,reproduction_rate,total_tests,new_tests,positive_rate,tests_per_case,tests_units
0,Afghanistan,2020-02-24,NaN,NaN,NaN,NaN,8.33,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,2020-02-25,NaN,NaN,NaN,NaN,8.33,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,2020-02-26,NaN,NaN,NaN,NaN,8.33,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,2020-02-27,NaN,NaN,NaN,NaN,8.33,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,2020-02-28,NaN,NaN,NaN,NaN,8.33,NaN,NaN,NaN,NaN,NaN,NaN
5,Afghanistan,2020-02-29,NaN,NaN,NaN,NaN,8.33,NaN,NaN,NaN,NaN,NaN,NaN
6,Afghanistan,2020-03-01,NaN,NaN,NaN,NaN,27.78,NaN,NaN,NaN,NaN,NaN,NaN
7,Afghanistan,2020-03-02,NaN,NaN,NaN,NaN,27.78,NaN,NaN,NaN,NaN,NaN,NaN
8,Afghanistan,2020-03-03,NaN,NaN,NaN,NaN,27.78,NaN,NaN,NaN,NaN,NaN,NaN
9,Afghanistan,2020-03-04,NaN,NaN,NaN,NaN,27.78,NaN,NaN,NaN,NaN,NaN,NaN


In [165]:
dataset.tail(10)

,location,date,icu_patients,hosp_patients,weekly_icu_admissions,weekly_hosp_admissions,stringency_index,reproduction_rate,total_tests,new_tests,positive_rate,tests_per_case,tests_units
127807,Zimbabwe,2021-10-23,NaN,NaN,NaN,NaN,44.44,0.72,1350237.0,3183.0,0.013,78.7,tests performed
127808,Zimbabwe,2021-10-24,NaN,NaN,NaN,NaN,44.44,0.74,1351732.0,1495.0,0.014,71.9,tests performed
127809,Zimbabwe,2021-10-25,NaN,NaN,NaN,NaN,44.44,0.74,1355031.0,3299.0,0.016,61.9,tests performed
127810,Zimbabwe,2021-10-26,NaN,NaN,NaN,NaN,44.44,0.75,1357443.0,2412.0,0.017,60.3,tests performed
127811,Zimbabwe,2021-10-27,NaN,NaN,NaN,NaN,44.44,0.75,1361473.0,4030.0,0.018,56.8,tests performed
127812,Zimbabwe,2021-10-28,NaN,NaN,NaN,NaN,44.44,0.76,1365439.0,3966.0,0.015,65.5,tests performed
127813,Zimbabwe,2021-10-29,NaN,NaN,NaN,NaN,44.44,NaN,1368179.0,2740.0,0.016,62.5,tests performed
127814,Zimbabwe,2021-10-30,NaN,NaN,NaN,NaN,44.44,NaN,1371934.0,3755.0,0.016,64.2,tests performed
127815,Zimbabwe,2021-10-31,NaN,NaN,NaN,NaN,44.44,NaN,NaN,NaN,NaN,NaN,NaN
127816,Zimbabwe,2021-11-01,NaN,NaN,NaN,NaN,44.44,NaN,NaN,NaN,NaN,NaN,NaN


All of the numerical variables are represented by `pandas` library as floats64. This is fine because there are a lot of missing values and soon we will replace them with mean or median, depending on the skewness, and these will not be integer numbers. 

Let's for now take a look at all unique values from the `tests_units` column.

This variable since its not numerical, it might have a weak relation to the other numerical variables and will not help much further when the prediction will be made.

In [166]:
dataset['tests_units'].value_counts()
dataset.drop('tests_units', axis='columns', inplace=True)

/home/ion/.local/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


(127167, 12)